In [32]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [125]:
path = "/gdrive/My Drive/ML:March2020/Assignments/data/"
data = pd.read_csv(path+"chennai_house_multivariate_train.csv")
data.head(10)

,AreaCode,INT_SQFT,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,OTHER_ROOMS,PARK_FACILITY,BUILDINGTYPE,UTILITY_AVAIL,STREET,MZZONE,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,REG_FEE,COMMIS,SALES_PRICE
0,4,1004,131,1,1,1,1,1,1,2,1,4.0,3.9,4.9,380000,144400,7600000
1,2,1986,26,2,1,2,0,1,1,1,4,4.9,4.2,2.5,760122,304049,21717770
2,1,909,70,1,1,1,1,1,2,1,5,4.1,3.8,2.2,421094,92114,13159200
3,7,1855,14,3,2,0,0,3,3,2,3,4.7,3.9,3.6,356321,77042,9630290
4,4,1226,84,1,1,1,1,3,1,1,2,3.0,2.5,4.1,237000,74063,7406250
5,3,1220,36,2,1,1,0,1,3,0,4,4.5,2.6,3.1,409027,198316,12394750
6,3,1167,137,1,1,1,0,3,1,0,5,3.6,2.1,2.5,263152,33955,8488790
7,7,1847,176,3,2,0,0,1,1,1,6,2.4,4.5,2.1,604809,235204,16800250
8,3,771,175,1,1,0,0,3,3,2,6,2.9,3.7,4.0,257578,33236,8308970
9,7,1635,74,2,1,1,0,3,2,0,3,3.1,3.1,3.3,323346,121255,8083650


In [126]:
data.shape

(5999, 17)

In [127]:
Xcol=data.drop(columns=["SALES_PRICE"])
Ycol=data["SALES_PRICE"]

In [128]:
X=np.array(Xcol)
Y=np.array(Ycol)
Y=np.reshape(Y,(-1,1))
X.shape,Y.shape

((5999, 16), (5999, 1))

In [129]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.25)

In [130]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4499, 16), (1500, 16), (4499, 1), (1500, 1))

In [131]:
import math
def E_D(point1,point2):
  sum_squared_distance = 0
  for i in range(len(point1)):
    sum_squared_distance += math.pow(point1[i] - point2[i], 2)
  return math.sqrt(sum_squared_distance)

In [132]:
from scipy import stats
import math
def knn(data, input_X, k, distance_fn):
  #create a list to store all predictions 
  predictions=[]   

  #for loop to make predictions for every sample in the input
  for i in range(len(input_X)):
    #store all the distances between the input and the training data
    #along with the index of the training data
    neighbor_distances_and_indices = []

    for index, train in enumerate(data): 
      #find the distance between the new input with every sample in the data
      distance = distance_fn(train[:], input_X[i])   

      #store the distance and the index wrt the training samples   
      neighbor_distances_and_indices.append((distance, index)) 

    #sort the smallest to largest distance
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    #pick the first K distances
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k] 
    
    # find the value of y in the training data for the first k distances
    k_nearest_labels = [Y_train[i][0] for distance, i in k_nearest_distances_and_indices]
    
    # find the most common value of y 
    prediction=(stats.mode(k_nearest_labels))

    #append the predicited value to the total_predictions list
    predictions.append(prediction.mode[0])
      
  return predictions # returns the list of all predictions for a given input

In [133]:
clf_query = X_test[:1]
Y_pred = knn(X_train, clf_query, k=5, distance_fn=E_D)
print(Y_pred)

[12364620]


In [134]:
clf_query = X_test
Y_pred = knn(X_train, clf_query, k=5, distance_fn=E_D)

In [135]:
Y_train = Y_train.ravel()

In [136]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(algorithm = 'brute',n_neighbors=4)
regressor.fit(X_train, Y_train)

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                    weights='uniform')

In [137]:
Y_pred = regressor.predict(X_test)

In [138]:
from sklearn.metrics import r2_score
r2_score(Y_test, Y_pred)

0.7341854201754062